<a href="https://colab.research.google.com/github/Alfrermn/g20_p1_aprendizaje_automatico/blob/a/modelo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO FINAL

EJERCICIO 8

c)

In [49]:
# Primeramente leemos los datos y los guardamos en dos variables distintas, disp_df y comp_df, con los datos de entrenamiento y pruebas respectivamente.
import pandas as pd
disp_df = pd.read_csv("/disp_st20ns1.txt.bz2",
 compression="bz2",
 index_col=0)
comp_df = pd.read_csv("/comp_st20ns1.txt.bz2",
 compression="bz2",
 index_col=0)

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import time
import pickle

x = disp_df.drop('salida', axis=1)
y = disp_df['salida']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.16666666, random_state=20)

#Entrenamos el modelo con los mejores hiperparámetros en el conjunto de entrenamiento
modelo_final = LinearRegression(copy_X= True, fit_intercept= True, n_jobs= -1, positive= True)

modelo_final.fit(x_train, y_train)

with open('/modelo_final.pkl', 'wb') as file:
    pickle.dump(modelo_final, file)


Apartado d

In [52]:
import pandas as pd
import numpy as np
import pickle

# Cargar el modelo final desde el archivo modelo_final.pkl
with open('/modelo_final.pkl', 'rb') as f:
    model = pickle.load(f)

# Obtener las predicciones utilizando el modelo final
predicciones = model.predict(comp_df)

# Crear un DataFrame con las predicciones y guardarlas en un archivo CSV llamado "predicciones.csv"
pred_df = pd.DataFrame(predicciones, columns=['salida'])
pred_df.to_csv('/predicciones.csv', index=False)

